In [67]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

In [68]:
data = pd.read_csv("C:/Users/yayat/OneDrive/Documentos/Jeimmy/Universidad/NPL/Taller 5/bob_esponja.csv")
data

,Personaje,Descripción
0,Bob Esponja Pantalones Cuadrados,Bob Esponja es una esponja de mar con forma re...
1,Patricio Estrella,"Es una estrella de mar de color rosa, el mejor..."
2,Calamardo Tentáculos,Calamardo Tentáculos es un personaje principal...
3,Arenita Mejillas,Arenita Mejillas (Sandy Cheeks en inglés) en u...
4,Eugenio H. Cangrejo,Don Eugenio H. Cangrejo (normalmente llamado S...
5,Sheldon J. Plankton,"Sheldon J. Plankton, o simplemente Plankton, e..."
6,Karen Plankton,Karen Plankton es uno de los dos principales a...
7,Perlita Cangrejo,"Perla ""Perlita"" Cangrejo es un personaje princ..."
8,Sra. Puff,La Señora Puff es un personaje principal de Bo...
9,Gary el Caracol,Gary el Caracol es un personaje principal de B...


In [71]:
def pre_procesado(Descripción):
    Descripción = Descripción.lower()
    Descripción = re.sub(r"[\W\d_]+", " ", Descripción)
    Descripción = [palabra for palabra in Descripción.split() if palabra not in stopwords_sp]
    Descripción = [spanishStemmer.stem(palabra) for palabra in Descripción]
    Descripción = " ".join(Descripción)
    return Descripción

data['pre-procesado'] = data['Descripción'].apply(lambda Descripción: pre_procesado(Descripción))
        
data

,Personaje,Descripción,pre-procesado
0,Bob Esponja Pantalones Cuadrados,Bob Esponja es una esponja de mar con forma re...,bob esponj esponj mar form rectangul color ama...
1,Patricio Estrella,"Es una estrella de mar de color rosa, el mejor...",estrell mar color ros mejor amig bob esponj ju...
2,Calamardo Tentáculos,Calamardo Tentáculos es un personaje principal...,calamard tentacul personaj principal bob espon...
3,Arenita Mejillas,Arenita Mejillas (Sandy Cheeks en inglés) en u...,arenit mejill sandy cheeks ingles personaj pri...
4,Eugenio H. Cangrejo,Don Eugenio H. Cangrejo (normalmente llamado S...,don eugeni h cangrej normal llam señor cangrej...
5,Sheldon J. Plankton,"Sheldon J. Plankton, o simplemente Plankton, e...",sheldon j plankton simplement plankton dos ant...
6,Karen Plankton,Karen Plankton es uno de los dos principales a...,kar plankton dos principal antagon bob esponj ...
7,Perlita Cangrejo,"Perla ""Perlita"" Cangrejo es un personaje princ...",perl perlit cangrej personaj principal bob esp...
8,Sra. Puff,La Señora Puff es un personaje principal de Bo...,señor puff personaj principal bob esponj maest...
9,Gary el Caracol,Gary el Caracol es un personaje principal de B...,gary caracol personaj principal bob esponj que...


In [76]:
tfidf_vec = TfidfVectorizer()
tfidf = tfidf_vec.fit_transform(data['pre-procesado'].values)

tfidf_matrix = pd.DataFrame(tfidf.toarray())
tfidf_matrix.columns = tfidf_vec.get_feature_names()
tfidf_matrix.index = data.index

tfidf_matrix = tfidf_matrix.T

tfidf_matrix.round(3)

,0,1,2,3,4,5,6,7,8,9
abaj,0.057,0.00,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000
aborrec,0.000,0.00,0.091,0.000,0.000,0.0,0.000,0.0,0.000,0.000
acab,0.000,0.00,0.091,0.000,0.000,0.0,0.000,0.0,0.000,0.000
accident,0.049,0.00,0.000,0.000,0.045,0.0,0.000,0.0,0.000,0.000
acept,0.000,0.00,0.000,0.000,0.000,0.0,0.064,0.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...
viv,0.000,0.05,0.060,0.108,0.070,0.0,0.000,0.0,0.000,0.000
vol,0.057,0.00,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000
voz,0.195,0.00,0.000,0.139,0.000,0.0,0.000,0.0,0.000,0.000
vuelv,0.172,0.00,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000


In [73]:
# Distancia Coseno

In [74]:
dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos,columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.826466,0.918853,0.838917,0.868554,0.872613,0.938901,0.886072,0.857937,0.924381
1,0.826466,0.000000,0.791198,0.841713,0.831869,0.966669,0.930557,0.891881,0.952574,0.895000
2,0.918853,0.791198,0.000000,0.920036,0.831488,0.917784,0.950219,0.967358,0.968076,0.852161
3,0.838917,0.841713,0.920036,0.000000,0.908027,0.967723,0.948562,0.962264,0.972093,0.900968
4,0.868554,0.831869,0.831488,0.908027,0.000000,0.687797,0.892102,0.796912,0.895877,0.920780
5,0.872613,0.966669,0.917784,0.967723,0.687797,0.000000,0.795121,0.922782,0.986743,0.969443
6,0.938901,0.930557,0.950219,0.948562,0.892102,0.795121,0.000000,0.966336,0.984939,0.965513
7,0.886072,0.891881,0.967358,0.962264,0.796912,0.922782,0.966336,0.000000,0.873928,0.969604
8,0.857937,0.952574,0.968076,0.972093,0.895877,0.986743,0.984939,0.873928,0.000000,0.964203
9,0.924381,0.895000,0.852161,0.900968,0.920780,0.969443,0.965513,0.969604,0.964203,0.000000


In [ ]:
# cuales son los personajes mas parecidos?
Eugenio H. Cangrejo con Sheldon J. Plankton

In [ ]:
# Cuales son los personajes mas diferentes?
Sheldon J. Plankton con Sra. Puff